In [ ]:
# Note: Before starting the slideshow, make sure you run this code as it
# provides helper functions that the other slides need...
#
# Other than that, you can ignore this content as it won't show up in the slideshow.
#
# It's a helper function that makes it easier to show OpenCV images directly
# in the notebook environment. When using OpenCV locally, you'll want to use
# `cv2.imshow(name, img)` instead.

from scipy.spatial import distance as dist
import numpy as np
import cv2
import math

# Notebook setup + convenience functions
%matplotlib inline
import matplotlib.pyplot as plt

def force_bgr(img):
    '''Forces image to 3-channel representation if grayscale'''
    if len(img.shape) == 2 or img.shape[2] == 1:
        return cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    return img

def imshow(*args):
    '''Helper function to show images, because matplotlib and OpenCV aren't a perfect match'''
    fig = plt.figure()
    for i, img in enumerate(args):
        fig.add_subplot(1,len(args),i+1)
        plt.imshow(cv2.cvtColor(force_bgr(img), cv2.COLOR_BGR2RGB))

def blur(src, radius):
    ksize = int(2 * round(radius) + 1)
    return cv2.blur(src, (ksize, ksize))


# <center>Image processing using</center>

## <center>OpenCV + Python</center>

<br/>
<center><b>Tim Winters</b><center>
<center>Created by Dustin Spicuzza (Team 2423/1418)</center>
<br/>
<center>September 10, 2016</center>
<br/>
<center><span style="color: #ababab">NE FIRST University Day</span></center>

# Agenda

* Why OpenCV + Python?
* Image filtering demo
* pynetworktables

# Image processing

* FRC Teams do it a lot of ways
    * NIVision (LabVIEW)
    * GRIP (Uses OpenCV as engine)
    * OpenCV (various custom stuff)
    
* We're going to talk about OpenCV

# Why OpenCV?

* Originally developed by Intel
* It has thousands of image processing related algorithms and functions available
  * Highly optimized and reliable
  * Has building blocks that fit together
* Lets you do complex image processing without needing to understand the math
  * If you understand the math, it helps!

# Why OpenCV?

* Bindings for multiple languages
    * C/C++
    * Java
    * Python
* Multiple platforms supported
    * Windows
    * Linux
    * OSX
    * Android
* Oh, and it’s **FREE**!

# What OpenCV Provides

* Image I/O:
    * Read/Write images from disk
    * Use native OS functionality to interface with cameras
* Image Segmentation
    * Edge finding
    * Contour detection
    * Thresholding

# What OpenCV Provides

* Face detection
* Motion tracking
* Stereo vision support
* Support for GPU acceleration
* Machine learning operations
    * Classifiers
    * Neural networks

# What OpenCV Provides

* Distributed with lots of useful samples that you can use to figure out how OpenCV works
    * Face detection
    * Edge finding
    * Histograms
    * Square finder

Lots and lots and lots of stuff… 

# Why Python + OpenCV?

* Python is really easy to learn and use
    * Simple syntax
    * Rapid prototyping
* Most of the compute intensive work is implemented in C/C++
    * Python is just glue, realtime operation **is** possible
* NumPy is awesome
    * Manipulating image data is trivial compared to other OpenCV bindings (Java, C++)

# <center>Time to CODE!</center>

<center>Go to http://goo.gl/nB0NCG</center>

# About this environment

http://goo.gl/nB0NCG

* It's a Jupyter Notebook (formerly IPython Notebook)
    * This slideshow uses Jupyter too!
    
* It allows you to mix text and executable code in a webpage
* You execute each cell using SHIFT-ENTER

# Hello World!

* Click the cell with the following text, and press SHIFT-ENTER

In [ ]:
print("Hello class")

# Next Steps

* Execute the helper code

* The next cell tells you about the images available in your environment

In [ ]:
%ls images

# Hello image!

* Let's load an image and show it

In [ ]:
# Change this to load different images
img = cv2.imread('images/2016-cmp-5.jpg')
imshow(img)

# Hello image!

* You can show multiple images next to each other

In [ ]:
imshow(img, img)

# OpenCV Image Basics

* Images are stored as multidimensional arrays
    * Color images have 3 dimensions: height, width, channel
* Each pixel is a number stored in the array
* Numpy array notation allows you to do operations on individual pixels or ranges of pixels

In [ ]:
img[50, 150, :]           # Access a single pixel,

In [ ]:
x = img[24:42, 42:100, :]    # Access a range of pixels

# OpenCV Image Basics

* Color is represented by storing combinations of Red, Blue, and Green pixels in separate channels
    * OpenCV uses BGR representation, not RGB
* The amount of each individual color is represented in the individual channel
    * ‘dark’ is zero, ‘bright’ is 255
* Combine the channels to represent a color
    * Green = RGB( 0, 255, 0 )
    * Deep Pink = RGB( 255, 20, 147 )

# OpenCV Image Basics

* Using numpy we can easily fill an image with a single color

In [ ]:
# define image with height=240, width=320, 3 channels
shape = (240, 320, 3)
pink_img = np.empty(shape, dtype=np.uint8)

# Fill every pixel with a single color
pink_img[:] = (147, 20, 255)

imshow(pink_img)

# Practical Example

* 2016 FIRST Stronghold: find targets that are surrounded by retroreflective tape, and shoot boulders into them
* 2017 FIRST Steamworks: use two targets to align to gear holder on the ship

# Practical Example
* Finding gray tape at a distance isn’t particularly easy
    * Key part of image processing is removing as much non-essential information from image 
* We can do better!

# Retroreflective Tape

* It has a useful property -- it reflects light directly back at the source
* What can we do with this property?

* Shine bright LEDs at the target and the tape reflects that color back to the camera
* Reduce exposure of camera so only bright light sources are seen 

# What Color Light?

* Humans are most sensitive to green light, and cameras mimic this
 * Twice as many green sensors as red or blue

This makes green light best suited for vision

# A note about exposure

* Webcams support setting the exposure manually (yay)
* Some cameras only allow particular exposure settings 
    * The lifecam is one of them
* OpenCV has bugs, it doesn't set the exposure properly
* Here's a workaround that works on linux:

```sh 
v4l2-ctl -d /dev/video0 -c exposure_auto=1 -c exposure_absolute=10
```

# Retroreflective Tape

In [ ]:
img1 = cv2.imread('images/2016-p0.jpg')
img2 = cv2.imread('images/2016-p1.jpg')
imshow(img1)
imshow(img2)

# Practical Example

Processing steps to find targets:

* Isolate the green portions of the image
* Analyze the green portions to determine targets

**Note**: There are a lot of ways to go about this, I'm just showing you one way

# Blurring the Image

  * Get rid of small artifacts (noise)
  * Makes the target a little more complete
    * Easier for thresholding and contour finding

In [ ]:
blurred = blur(img2, 2)
imshow(blurred)

# Identify the green

* What is “green” anyways?
    * <span style="color: green">This is green.</span> <span style="color: darkgreen">This is also green.</span>
* To a computer, green is really a range of colors
* An object’s color changes depending on lighting conditions
* We can transform the image to identify colors independent of lighting conditions

# Identify the green

* Convert the image from RGB to HSV
    * Hue: the color
    * Saturation: Colorfulness
    * Value: Brightness

In [ ]:
hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
imshow(img2)
imshow(hsv)

# Identify the green

That doesn't show why HSV is useful. Let's look at the individual channels instead.

In [ ]:
h, s, v = cv2.split(hsv)
imshow(h)
imshow(s)
imshow(v)

# Identify the green

* Green is a range of values present in the image
* ‘Threshold’ the image to get rid of the colors that we don’t care about
* Lots of ways to do this
    * Manually specify values
    * Automated methods

# Identify the green

`cv2.inRange` can threshold an image given two ranges of pixels.
* Wanted values are converted to 255
* Unwanted values are now 0

In [ ]:
lower = np.array([0, 145, 80])
upper = np.array([255, 255, 255])

filtered = cv2.inRange(hsv, lower, upper)
imshow(img2)
imshow(filtered)

# Identify the green

Sometimes, you end up with holes in your output

In [ ]:
img3 = cv2.imread('images/2013-f0.png')
hsv3 = cv2.cvtColor(img3, cv2.COLOR_BGR2HSV)
blur3 = hsv3
blur3 = blur(hsv3, 2)

# Thresholds are different because different camera/lighting
lower3 = np.array([30, 175, 16])
upper3 = np.array([75, 255, 255])
filtered3 = cv2.inRange(blur3, lower3, upper3)
imshow(img3)
imshow(filtered3)

# Filling in the holes

* We can use a morphological operation to fill in the holes
    * Various types of morphology operations available
* They modify a pixel based on the values of its neighboring pixels
    * The one we use to fill in holes is a “closing” operation

In [ ]:
# kernel tells opencv how big a space to sample, and what shape to sample.
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2), anchor=(1,1))
# sample and close 3 times
output = cv2.morphologyEx(filtered3, cv2.MORPH_CLOSE, kernel,
                          iterations=3)
imshow(output)

# Identifying Targets

Use `findContours()` to find regions of interest
* Returns a list of points bounding each separate blob in the image (called a contour)
* Also returns a hierarchy so you can determine whether a contour is entirely inside another contour

In [ ]:
contours, _ = cv2.findContours(output, cv2.RETR_EXTERNAL,
                                    cv2.CHAIN_APPROX_SIMPLE)

# Identifying Targets

If you want to see what it found, you can draw the found contours.

In [ ]:
dst = np.zeros(shape=img3.shape, dtype=img3.dtype)
cv2.drawContours(dst, contours, -1, (0, 255, 255), 3)
imshow(dst)
#print(contours[0])

# Identifying Targets

* As you can see, contours aren't the whole story


# Identifying Targets

* Contour analysis
    * Discard non-convex contours
    * Convert to polygon approximation (approxPolyDP)
        * This will turn our multi-vertex contour into a contour with the vertex we see in the real world
    * Discard polygons that aren't the right size


# Magic?

In [ ]:
min_width = 20  # in pixels 
min_height = 50
results = []
centers = []

# Iterate over each contour
for cnt in contours:
    
    # Skip initial contours that don't meet requirements
    _, _, w, h = cv2.boundingRect(cnt)
    if w < h or w < min_width: # or len(a2) not in (4, 5):
        continue
    
    # Find approximation that contains 4 sides
    points_wanted = 4
    precision = 100
    approx = None
    for x in range(precision):
        epsilon = (x/precision)*cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)
        if approx.shape[0] == points_wanted:
            break

    # We only care about objects that are wider than they are tall, and things wider
    # than a particular width. Only keep things that meet that criteria.
    _, _, w, h = cv2.boundingRect(approx)
    if w < h or w < min_width or h < min_height: # or len(a2) not in (4, 5):
        continue
    
    results.append(approx.squeeze(1))
    M = cv2.moments(approx)
    if M["m00"] == 0:
        continue
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])

    # draw the contour and center of the shape on the image
    centers.append((int(cX), int(cY)))
    cv2.putText(dst, "center", (cX - 20, cY - 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    
dst2 = np.zeros(shape=img3.shape, dtype=img3.dtype)
cv2.drawContours(dst2, results, -1, (0, 255, 255), 3)
imshow(dst)
imshow(dst2)


# Contour Features
You can find more about contour features and properties [here](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_contours/py_table_of_contents_contours/py_table_of_contents_contours.html)

For 2020, we will want to heavily use the convex hull and 

# Magic?

In [ ]:
# Finally, draw out our results
for cnt in centers:
    cv2.circle(dst2, (cnt[0], cnt[1]), 1, (255, 255, 255), 5)
imshow(dst2)
#print(results[0])

# Identifying Targets

* Sometimes you need to do more work
    * Use ratios to determine which target you're looking at
    * Remove duplicates (inner rectangles)
    * Other types of validation

# Now what?

We have targets... probably should do something with them?

In [ ]:
# Lets sort the targets left to right, using the x value of the center
def getXValueOfCenter(cnta):
    M = cv2.moments(cnta)
    if M["m00"] == 0:
        return 0
    return int(M["m10"] / M["m00"])

results = sorted(results, key=lambda c: getXValueOfCenter(c))

# Image Points

This is the hardest part of the whole process. Singling out the chosen obejct points, and making sure they're in the right order.
OpenCV doesn't have a strict method of ordering contours, so we have to do some manual finding of the contours.
Luckily, we're dealing with squares so it's easy to figure this out.

In [ ]:
# Now lets sort the coordinates for each contour clockwise starting with top left

# taken from https://www.pyimagesearch.com/2016/03/21/ordering-coordinates-clockwise-with-python-and-opencv/
def order_points(pts):
    # sort the points based on their x-coordinates
    xSorted = pts[np.argsort(pts[:, 0]), :]
    # grab the left-most and right-most points from the sorted
    # x-roodinate points
    leftMost = xSorted[:2, :]
    rightMost = xSorted[2:, :]
    # now, sort the left-most coordinates according to their
    # y-coordinates so we can grab the top-left and bottom-left
    # points, respectively
    leftMost = leftMost[np.argsort(leftMost[:, 1]), :]
    (tl, bl) = leftMost
    # now that we have the top-left coordinate, use it as an
    # anchor to calculate the Euclidean distance between the
    # top-left and right-most points; by the Pythagorean
    # theorem, the point with the largest distance will be
    # our bottom-right point
    D = dist.cdist(tl[np.newaxis], rightMost, "euclidean")[0]
    (br, tr) = rightMost[np.argsort(D)[::-1], :]
    # return the coordinates in top-left, top-right,
    # bottom-right, and bottom-left order
    return np.array([tl, tr, br, bl], dtype="int32")

results = np.array(list(map(order_points, results)))
print(results)


# Calculate angle/distance to target

* If we know where the object is in the real world, we can map out the camera position using the object's position in the image
* Slight adjustments for inaccuracies in the camera are needed

* We need a couple parts for this
    * Camera Calibration Matrix
    * Camera Distortion Coefficients
    * Object Points
    * Image Points


# Camera Calibration Matrix
OpenCV has functions built in to get your camera's calibration numbers

Luckily, you don't actually have to understand the code to get your calibration, but you can find the code [here](https://github.com/opencv/opencv/blob/master/samples/python/calibrate.py)

To begin, we need ~25 images taken with the camera of a checkerboard pattern. Laser cutting onto some 1/4" project panels works well, since this will give you a nice rigid surface

To generate the matrix from the raspberry pi, use the `cameraCapture.py` file in the repo.

# Camera Distortion Coefficients

If you've ever seen a camera that has a fisheye lens, then you know what lens distortion looks like.
This makes it hard to get good distance calculations, so we need to know how distorted the image is.
Luckily, the camera calibration gives us both the intrinsic properties of the camera, as well as the 
distortion coefficients, so we're done!

# Object Points

Perspective-n-Point works by mapping the 2d pixel coordinates to 3d coordinates in the real world.
In order to map these, we need to define which points we're going to reference. For 2020, the target is a half hexagon
We can use the four outer vertices as our points, with the center of the hexagon as our 'origin'.
To calculate the 3d coordinates for the 4 points, it is helpful to create a CAD drawing and use the measurement tools.

Take some time now and decide what points you'd like to use for vision targeting. You need at least 4, and the points should
  easy to distinguish.

# Object Points for 2013 Targets

We can look at the field diagram to generate the object points. I modelled it in fusion360 and came up with the following. Origin is at the center of the high goal.


In [ ]:
# Store object points

obj_points = np.array([(-103, 3.5, 0),
                        (-41, 3.5, 0),
                        (-41, -25.50, 0),
                        (-103, -25.50, 0),
                        (-31, 10, 0),
                        ( 31, 10, 0),
                        (31, -10, 0),
                        (-31, -10, 0),
                        (41, 3.5, 0),
                        (103, 3.5, 0),
                        (103, -25.50, 0),
                        (41, -25.50, 0)], dtype=np.float32)

# made up matrix using 4.4mm focal length found online
cam_matrix = np.array([[.17, 0, 320],
                       [0, .17, 240],
                       [0,  0,  1]], dtype=np.float)

# pretending no distortion
dist_coeffs = np.array([1.3287735059062630e-01, -6.8376776294978103e-01, 0., 0., 8.1215478360827675e-01])
# flatten list of points
results = results.reshape(12, 2).astype('float32')
axis = np.float32([[24,0,0], [0,24,0], [0,0,-24]]).reshape(-1,3)
print(results)
print(obj_points)

# Solve PnP
The last part of the problem is solving the PnP equation by calling a single method

```
ret, rvecs, tvecs = cv2.solvePnP(object_points, image_points, cam_matrix, dist_coeffs)
```

`rvecs` will be a vector of the rotations in radians
`tvecs` will be a vector of the translations in whatever unit you specify for the object points. This tells us the 
  distance to the target

In [ ]:
# THIS IS THE GOD FUNCTION
ret, rvecs, tvecs = cv2.solvePnP(obj_points, results, cam_matrix, dist_coeffs)

print("Translation Vectors")
print(tvecs)
# function used to draw 
def draw(img, imgpts):
    corner = centers[2]
    img = cv2.line(img, corner, tuple(imgpts[0].ravel()), (255, 0, 0), 5)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel()), (0, 255, 0), 5)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel()), (0, 0, 255), 5)
    return img


# this probably doesn't work because my camera numbers are off. But it would IRL!
imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, cam_matrix, dist_coeffs)
img = draw(dst2, imgpts.astype(np.int))
imshow(img)

# Now What?

* Send data via NetworkTables

* ... I forgot to write this slide. It's easy, I promise.

# Where to run the image processing

* RoboRIO
    * RoboRIO is relatively slow, OpenCV eats a lot of CPU
        * Hint: Make the images small (320x240)
    * Less hardware to deal with
    * FIRST intends to install OpenCV by default in 2017
    

# Where to run the image processing

* Driver Station
    * Streaming images to OpenCV is possible
        * Various latency bugs
    * Latency is an issue here
    * mDNS problems (hopefully will be resolved in 2017)

# Where to run the image processing

* Coprocessor (Jetson, Raspberry PI, Nexus 5)
    * Lots of teams do this
    * More hardware to deal with
    * Potentially higher fidelity processing

# Want code?

* Working OpenCV code integrated with mjpg-streamer
    * https://github.com/frc2423/2016/tree/master/OpenCV
    * Includes code for storing images onto USB drive during matches
    * Don't let our robot's performance fool you... :(
    
* The stuff we did here will be available sometime tonight
    * https://github.com/virtuald/frc-imageprocessing-workshop-2016

# If you want more

* Team 254 gave an excellent presentation at CMP in 2016
    * https://goo.gl/mppi4E
    * Video/audio: http://www.chiefdelphi.com/forums/showthread.php?t=147568&page=3
    * Latency compensation is an excellent technique presented here

# Resources

* Python 3.5.x
    * https://www.python.org/downloads/
* Learn Python
    * http://www.codecademy.com/tracks/python

* OpenCV 3.1.0
    * http://opencv.org

* NumPy
    * Official site: http://www.numpy.org

# Resources

* roborio-packages
    * https://github.com/robotpy/roborio-packages
* OpenCV for RoboRIO
    * https://github.com/robotpy/roborio-opencv
* mjpg-streamer for RoboRIO
    * https://github.com/robotpy/mjpg-streamer

# Resources

* pynetworktables
    * source code + examples @
      https://github.com/robotpy/pynetworktables

* Edit & debug python code using Eclipse
    * Pydev: http://pydev.org/

# One more thing...

FIRSTwiki: https://firstwiki.github.io

* Publicly editable repository of information related to FIRST Robotics
    * Technical topics
    * Non-technical
    * Team pages
* Add content to your team’s page!

# <center>Questions?</center>